In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [2]:
_root = "../../gru_ode_bayes/datasets/me/"
_N = pd.read_excel(_root + 'JSTdatasetR4.xlsx', sheet_name=1)

_N = _N.loc[_N['country'] == 'Japan']\
       .reset_index(drop=True)\
       .drop(['country', 'iso', 'ifs', 'peg_type', 'peg_base'], axis = 1)

In [3]:
_N

,year,pop,rgdpmad,rgdppc,rconpc,gdp,iy,cpi,ca,imports,...,eq_capgain,eq_dp,eq_capgain_interp,eq_tr_interp,eq_dp_interp,bond_rate,eq_div_rtn,capital_tr,risky_tr,safe_tr
0,1870,34437.000000,737.375497,3.263085,NaN,NaN,NaN,0.018112,-0.000029,0.000034,...,NaN,NaN,NaN,NaN,NaN,0.070806,NaN,NaN,NaN,NaN
1,1871,34648.000000,742.005311,3.283716,NaN,NaN,NaN,0.015191,-0.000010,0.000022,...,NaN,NaN,NaN,NaN,NaN,0.067468,NaN,NaN,NaN,NaN
2,1872,34859.000000,746.005336,3.301548,NaN,NaN,NaN,0.011103,-0.000017,0.000026,...,NaN,NaN,NaN,NaN,NaN,0.064384,NaN,NaN,NaN,NaN
3,1873,35070.000000,751.012261,3.323839,NaN,NaN,NaN,0.011270,-0.000016,0.000028,...,NaN,NaN,NaN,NaN,NaN,0.065359,NaN,NaN,NaN,NaN
4,1874,35235.000000,756.179935,3.346751,5.355500,NaN,NaN,0.013098,-0.000020,0.000023,...,NaN,NaN,NaN,NaN,NaN,0.066166,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,2012,127148.442941,23121.976878,101.989396,103.841504,494.9572,0.210520,105.448531,4.671442,79.645400,...,-0.064061,0.021815,NaN,NaN,NaN,0.008392,0.020418,0.013212,0.008020,0.018487
143,2013,126930.135827,24006.651071,104.179878,106.454567,503.1756,0.221191,105.810220,4.783683,91.739600,...,0.693687,0.016529,NaN,NaN,NaN,0.007000,0.027994,0.166683,0.304568,0.008847
144,2014,126717.809730,24560.644237,104.668370,105.688334,513.6980,0.214782,108.730582,3.863009,102.803300,...,0.086554,0.015922,NaN,NaN,NaN,0.005263,0.017300,0.057370,0.083423,0.027183
145,2015,126576.258998,25130.957579,106.057229,105.368373,529.9536,0.214665,109.592816,16.412609,95.280000,...,0.095425,0.014819,NaN,NaN,NaN,0.003467,0.016233,0.054087,0.091816,0.009271


46

In [8]:
NUM_COL = _N.columns.__len__()

_N = _N[[i for i in _N.columns[0:NUM_COL + 1]]].reset_index()

_Mask = _N[[i for i in _N.columns[2:NUM_COL+2]]].copy()

_N.rename(columns={'index':'ID', 'year':'Time'}, inplace=True)

for idx, col in enumerate(_N.columns[2:len(_N.columns)]):
    _N.rename(columns={col:f'Value_{idx}'}, inplace=True)
    

for idx, col in enumerate(_Mask.columns):
    _Mask.rename(columns={col:f'Mask_{idx}'}, inplace=True)

In [9]:
for i in _Mask.columns:
    _Mask[i].mask(~_Mask[i].isnull(), '1.0', inplace=True)
    
for i in _Mask.columns:
    _Mask[i].mask(_Mask[i].isnull(), '0.0', inplace=True)

In [10]:
_N

,ID,Time,Value_0,Value_1,Value_2,Value_3,Value_4,Value_5,Value_6,Value_7,...,Value_35,Value_36,Value_37,Value_38,Value_39,Value_40,Value_41,Value_42,Value_43,Value_44
0,0,1870,34437.000000,737.375497,3.263085,NaN,NaN,NaN,0.018112,-0.000029,...,NaN,NaN,NaN,NaN,NaN,0.070806,NaN,NaN,NaN,NaN
1,1,1871,34648.000000,742.005311,3.283716,NaN,NaN,NaN,0.015191,-0.000010,...,NaN,NaN,NaN,NaN,NaN,0.067468,NaN,NaN,NaN,NaN
2,2,1872,34859.000000,746.005336,3.301548,NaN,NaN,NaN,0.011103,-0.000017,...,NaN,NaN,NaN,NaN,NaN,0.064384,NaN,NaN,NaN,NaN
3,3,1873,35070.000000,751.012261,3.323839,NaN,NaN,NaN,0.011270,-0.000016,...,NaN,NaN,NaN,NaN,NaN,0.065359,NaN,NaN,NaN,NaN
4,4,1874,35235.000000,756.179935,3.346751,5.355500,NaN,NaN,0.013098,-0.000020,...,NaN,NaN,NaN,NaN,NaN,0.066166,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,142,2012,127148.442941,23121.976878,101.989396,103.841504,494.9572,0.210520,105.448531,4.671442,...,-0.064061,0.021815,NaN,NaN,NaN,0.008392,0.020418,0.013212,0.008020,0.018487
143,143,2013,126930.135827,24006.651071,104.179878,106.454567,503.1756,0.221191,105.810220,4.783683,...,0.693687,0.016529,NaN,NaN,NaN,0.007000,0.027994,0.166683,0.304568,0.008847
144,144,2014,126717.809730,24560.644237,104.668370,105.688334,513.6980,0.214782,108.730582,3.863009,...,0.086554,0.015922,NaN,NaN,NaN,0.005263,0.017300,0.057370,0.083423,0.027183
145,145,2015,126576.258998,25130.957579,106.057229,105.368373,529.9536,0.214665,109.592816,16.412609,...,0.095425,0.014819,NaN,NaN,NaN,0.003467,0.016233,0.054087,0.091816,0.009271


In [11]:
_N = _N.fillna(0).join(_Mask)

In [12]:
_N.to_csv('data.csv', index=False)

In [13]:
_N = pd.read_csv('data.csv')

In [14]:
_N

,ID,Time,Value_0,Value_1,Value_2,Value_3,Value_4,Value_5,Value_6,Value_7,...,Mask_35,Mask_36,Mask_37,Mask_38,Mask_39,Mask_40,Mask_41,Mask_42,Mask_43,Mask_44
0,0,1870,34437.000000,737.375497,3.263085,0.000000,0.0000,0.000000,0.018112,-0.000029,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,1871,34648.000000,742.005311,3.283716,0.000000,0.0000,0.000000,0.015191,-0.000010,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,1872,34859.000000,746.005336,3.301548,0.000000,0.0000,0.000000,0.011103,-0.000017,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,1873,35070.000000,751.012261,3.323839,0.000000,0.0000,0.000000,0.011270,-0.000016,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,1874,35235.000000,756.179935,3.346751,5.355500,0.0000,0.000000,0.013098,-0.000020,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,142,2012,127148.442941,23121.976878,101.989396,103.841504,494.9572,0.210520,105.448531,4.671442,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
143,143,2013,126930.135827,24006.651071,104.179878,106.454567,503.1756,0.221191,105.810220,4.783683,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
144,144,2014,126717.809730,24560.644237,104.668370,105.688334,513.6980,0.214782,108.730582,3.863009,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
145,145,2015,126576.258998,25130.957579,106.057229,105.368373,529.9536,0.214665,109.592816,16.412609,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [15]:
_full_idx = np.arange(_N["ID"].nunique())
_Nfolds = 5

np.random.seed(432)

for fold in range(_Nfolds):
    train_idx, test_idx = train_test_split(_full_idx, test_size=0.1)
    train_idx, val_idx = train_test_split(train_idx, test_size=0.2)
    _fold_dir = f"{_root}/datasets/me_{fold}/"
    if not os.path.exists(_fold_dir):
        os.makedirs(_fold_dir)
    np.save(_fold_dir+f"train_idx.npy",train_idx)
    np.save(_fold_dir+f"val_idx.npy",val_idx)
    np.save(_fold_dir+f"test_idx.npy",test_idx)